In [1]:
import numpy as np
import pandas as pd
from nltk import sent_tokenize
import re
import pymorphy2
import string
from tqdm import tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity
import operator
from collections import Counter
regex = re.compile('[%s]' % re.escape(string.punctuation))

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
def get_model(data, mode):
    if mode == 'collection':
        tokens = ' '.join(data).split()
    elif mode == 'doc':
        tokens = data.split()
    else:
        raise ValueError('invalid mode')
    collection_length = len(tokens)
    collection_model = Counter(tokens)
    for key in collection_model.keys():
        collection_model[key] /= collection_length
    return collection_model

In [75]:
def get_query_prob(query, lambda_coef, collection_model, doc_model):
    p = 1
    valid_prob = False
    for token in query.split():
        if token in collection_model:
            valid_prob = True
            collection_prob = collection_model[token]
            if token in doc_model:
                doc_prob = doc_model[token]
            else:
                doc_prob = collection_prob
            p *= ((1 - lambda_coef) * collection_prob + lambda_coef * doc_prob)
    if valid_prob:
        return p
    return 0

In [4]:
facts = ['Калифорнийский посёлок был независим от США почти 3 месяца пока не пришла пора праздновать День независимости и закупать алкоголь',
        'В наполеоновской артиллерии служили три генерала из одной семьи',
        'Совершивший первое кругосветное путешествие на велосипеде ездил по России на коне']
normalized_facts = []
for i in range(len(facts)):
    tokens = facts[i].split()
    for j in range(len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form
    normalized_facts.append(' '.join(tokens))

## Normalized facts 

In [5]:
for fact in normalized_facts:
    print(fact, '\n')

калифорнийский посёлок быть независимый от сша почти 3 месяц пока не пришлый пора праздновать день независимость и закупать алкоголь 

в наполеоновский артиллерия служить три генерал из один семья 

совершить первое кругосветный путешествие на велосипед ездить по россия на кон 



In [102]:
docs = open('data.txt', encoding='utf8').read()
docs = sent_tokenize(docs)
normalized_docs = []
for i in range(len(docs)):
    docs[i] = regex.sub(' ', docs[i])
    docs[i] = docs[i][:-1]
    tokens = docs[i].split()
    for j in range(len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form     
    tokens_without_stop_words = []
    stop_POS = ['PREP', 'CONJ', 'PRCL', 'NPRO']
    for token in tokens:
        POS = morph.parse(token)[0].tag.POS
        if POS not in stop_POS and POS is not None:
            tokens_without_stop_words.append(token)
    normalized_docs.append(' '.join(tokens_without_stop_words))

In [70]:
collection_model = get_model(normalized_docs, 'collection')
doc_models = [get_model(doc, 'doc') for doc in normalized_docs]

In [122]:
probs = {}
lambda_coef = 0.5
for i in range(len(doc_models)):
    probs[i] = get_query_prob(normalized_facts[1], lambda_coef, collection_model, doc_models[i])

In [124]:
sorted(probs.items(), key=operator.itemgetter(1), reverse=True)

[(106, 1.8209924515013523e-12),
 (116, 1.4040274787577706e-12),
 (172, 1.0544644872559073e-12),
 (126, 1.0264112974619427e-12),
 (121, 6.596554172308157e-13),
 (123, 2.3181409314966286e-13),
 (83, 1.4640863740250146e-13),
 (145, 9.014122063808291e-14),
 (20, 7.754511870308496e-14),
 (111, 7.754511870308494e-14),
 (7, 6.914771741308632e-14),
 (25, 5.4872135220088675e-14),
 (125, 5.4872135220088675e-14),
 (4, 5.2352914833089084e-14),
 (124, 5.0062714481271285e-14),
 (28, 3.975681289809115e-14),
 (136, 3.7533971380150344e-14),
 (76, 3.723759251109156e-14),
 (112, 3.6099880078253053e-14),
 (113, 3.6099880078253053e-14),
 (115, 3.6099880078253053e-14),
 (122, 3.555811225309184e-14),
 (151, 3.555811225309184e-14),
 (89, 3.3869560533833606e-14),
 (171, 3.044922788139012e-14),
 (174, 2.945091131491103e-14),
 (109, 2.927360548122191e-14),
 (91, 2.7582429281149742e-14),
 (88, 2.7160710963093224e-14),
 (170, 2.515311819124005e-14),
 (107, 2.4722755749867817e-14),
 (150, 2.3561824695950953e-14),
 

In [125]:
normalized_docs[106]

'оба сын быть генерал артиллерист наполеоновский армия'